# Chapter 3: Risk

## 风险的定义：用标准差，但成本用方差

第三章为整本书定风险的调子:
- 衡量风险的metrics: standard deviation, 因为它很直观，而且无论是单个资产还是投资组合都可计算
- 风险要作为一种成本，纳入决策过程。成本正比于variance

这样，$N$个风险资产$\{r_i: i=1,2,\dots,N\}$的risk profile完全由它的covariance matrix刻画
$$
\Sigma = \left[Cov \left(r_i,r_j\right) \right]
$$

投资组合 $h_P\in \mathbb{R}^N$的风险是
$$
\sigma_P = \sqrt{h_P^\top \Sigma h_P}
$$
相应的风险成本是
$$
\sigma_P^2 = h_P^\top \Sigma h_P
$$

### 为什么选择方差: 平方级惩罚重复下注的风险

根据方差的公式:
$$
\sigma_P^2 = h_P^\top \Sigma h_P = \sum_{i,j}h_{P,i}h_{P,j}\rho_{i,j}\sigma_i\sigma_j
$$
如果两个资产高度相关$\rho_{i,j}\to 1$, 它们带来的风险会以乘法级别上升。
方差作为风险的成本，让投资组合天然偏好diversification

## 主动管理视角: active/residual risk

第三章最“APM”的一刀：传统 MPT 在总风险/总收益里打转，但机构投资者（也就是本书的默认对象）更关心相对基准的主动风险与主动收益。

这会直接导向两个风险概念：
- Active risk / tracking error：来自持仓偏离 benchmark 的那部分风险（关键由 active position 的规模决定，而不是 benchmark 自己多大）。数学上来说，就是$$(h_P - h_B)^\top \Sigma (h_P - h_B)$$
- Residual risk：把组合收益对 benchmark 做线性解释后，剩下那部分不相关的波动（后面会和 alpha/IR 紧密绑定）。数学上来说是$$\sigma_P^2 - \beta^2_P \sigma_B^2$$

## 风险模型：从“直接估计 $\Sigma$”走向"结构化降维"

第三章在“模型”上做了一个非常实务的转折：你当然可以用历史收益直接估$\Sigma$，但维度太高(需要估计的参数随资产数目呈平方增长)、噪声太大；所以要用结构化风险模型降低维度、提高可预测性。

### 初等风险模型

书里先讲三种“初级风险模型”（这段很像在铺垫为什么需要 factor model）, 但是每个初等模型各有各的毛病：
- 单因子（market + residual，且假设 residual 不相关）：假设前后矛盾，光用市场因子无法得到互不相关的specific returns
- 全相关常数模型: 严重underfitting
- 全协方差历史模型: 参数远大于数据量，估计值方差太大 ￼

### 多因子风险模型

作者在 Technical Appendix 里把“真正要用的结构模型”写成一个多因子模型

#### 多因子风险模型假设: 
1. 资产的收益率$r_i$可以分解成因子收益率和特质收益率$u_i$
2. 因子本身的收益率$b$是需要估计的参数
3. 资产对因子的暴露（exposures）$X_i$是已知的数据
4. 不同资产的特质收益率不相关
5. 特质收益率和因子收益率正交
$$
r_i = X_i b + u_i
$$
因为因子个数往往远小于资产数目，因此可以对cross-section数据使用最小二乘法估计因子的收益率$b$和每个资产的特质收益率$u_i$。

并由此得到（你后面做任何组合风险/TE 计算都靠它）：
- 组合总方差：$$\sigma_P^2 = x_P^\top F x_P + h_P^\top A h_P$$（因子部分 + 特质部分）
- 主动风险（tracking error）：用 active holdings $h_{P,a}=h_P-h_B$ 同样分解出因子与特质贡献

其中$F$是 factor covariance matrix，$A$是 specific risk matrix。书里还强调：预测$F$本身就是一门大学问（权重、贝叶斯先验、条件异方差/Engle 等）。

这一步的本质是：
把“估一个巨大噪声矩阵 $\Sigma$”变成“估一个低维的 $F$ + 一个可控的 $A$”，并且能解释风险来源。

#### 什么样的因子是好的风险因子

> 好的风险因子，是那些能稳定、简洁、可解释地解释资产间“共同波动”的变量，而不是用来制造 alpha 的变量。

风险因子不是给你带来超额收益的，而是:
1. 能解释“共同波动”（common variation）
    - 资产之间为什么会一起涨、一起跌？
    - 对很多资产同时有显著 loading
    - 是相关性/协方差的主要来源
2. 结构稳定（structural stability）
    - 在时间上稳定
	- 在样本外仍然存在
	- 不依赖于“当期 regime 恰好如此”
3. 可解释、可命名（economic intuition）
    - 你能用一句话解释它代表什么风险
	- 投资委员会/风险委员会能理解
	- 在极端情形下，你能预判它会发生什么
4. 与 alpha 尽量正交
    - 不承载你“有 edge 的信息”
    - 你不想因为它获得收益，只想neutralize exposure
5. 低维、可管理（parsimony）
    - 多一个因子，协方差估计难度指数级上升
    - 宁可少而粗，不要多而碎
    - 风险模型追求 robustness，不是拟合度

一个典型的商品交易场景：

| 候选变量                  | 更像风险因子？ | 理由           |
|---------------------------|----------------|----------------|
| volatility                | ✅ 是           |  系统级波动来源，没有预测优势   |
| momentum                  | ✅ 是           |  系统级舆情，没有预测优势              |
| liquidity                 | ✅ 是           |  系统级流动性，没有预测优势              |
| 天然气价格                | ✅ 是           | 系统性边际成本 |
| 碳排放价格                | ✅ 是           | 结构性政策风险 |
| 电力负荷（climatology）   | ✅ 是           | 大尺度需求     |
| 极端寒潮/热浪预测误差     | ⚠️ 偏 alpha     | 信息优势       |
| 你自己的天气模型 residual | ❌ 不该         | 私有 edge      |



## 风险模型的“用途观”：不仅算风险，还要拆 bet

第三章的总结段落非常关键：结构化风险模型的价值不只是给你一个数字，而是能把风险拆成total vs active、market/benchmark vs residual、model vs specific、intentional vs incidental，用于事前预测（组合构建）、事中监控、事后归因。 